## Match the BCGs of two catalogs by proximity method. 
The BCGs in the redM_halos_fullmatchingparams_matchingv1.2_Buzzardv1.6.data catalog have wrong halo IDs. This is from expressing long integer IDs with i4, creating overflow error. This includes the GALAXY ID and HALOID. Everything else about the BCGs, including location and matched halos should be okay as they are not affected by this overflow error problem.

The BCGs in the catalog BCG_v1.6_info.data have correct IDs.

Patch the IDs in BCG_v1.6_info.data onto the fullmatchingparams catalog through (RA, DEC,Z) matching. There should be a one-to-one correspondance. 


In [1]:
import numpy as np
import sys
sys.path.append('/home/zzbenjamin94/Desktop/Astronomy/Research/DES_Galaxy_Cluster')

# import pyplot and set some parameters to make plots prettier
import matplotlib.pyplot as plt
from tools.plot_utils import plot_pretty
import pickle
plot_pretty()

# setup notebook for inline figures
%matplotlib inline

from tools.setup.setup import data_home_dir, home_dir, bigdata_home_dir
import astropy.io.fits as fits

datadir = data_home_dir()
homedir = home_dir()
bigdatadir = bigdata_home_dir()

/home/zzbenjamin94/Desktop/Astronomy/Research/DES_Galaxy_Cluster/data/
/data/DES/Cluster/


## Import the redM matched BCG/halo catalog

In [11]:
#Miscentering cut
import pickle

redM_halos = homedir + 'output/buzzard/redM_halos_fullmatchingparams_matchingv1.2_Buzzardv1.6.data'

infile = open(redM_halos,'rb')
redM_halos = pickle.load(infile)
infile.close()

redM_halos = redM_halos.view(np.recarray)
print redM_halos.dtype

#Change record array dtype. 
redM_halos = redM_halos.astype([('GALAXY_ID', int), ('HALOID', int), ('DESCID', '>i8'), \
                   ('RA_OBS', '>f8'), ('DEC_OBS', '>f8'), ('TRA', '>f8'), ('TDEC', '>f8'), \
                   ('Z', '>f8'), ('MVIR', '>f8'), ('VMAX', '>f8'), ('VRMS', '>f8'), ('RVIR', '>f8'), \
                   ('RS', '>f8'), ('NP', '>f8'), ('PX', '>f8'), ('PY', '>f8'), ('PZ', '>f8'), \
                   ('VX', '>f8'), ('VY', '>f8'), ('VZ', '>f8'), ('JX', '>f8'), ('JY', '>f8'), ('JZ', '>f8'), \
                   ('SPIN', '>f8'), ('RS_KLYPIN', '>f8'), ('MVIR_ALL', '>f8'), ('M200B', '>f8'), \
                   ('M200C', '>f8'), ('M500C', '>f8'), ('M2500C', '>f8'), ('XOFF', '>f8'), ('VOFF', '>f8'), \
                   ('SPIN_BULLOCK', '>f8'), ('B_TO_A', '>f8'), ('C_TO_A', '>f8'), ('PID', '>i8'), \
                   ('LUMTOT', '>f8'), ('LUM20', '>f8'), ('LBCG', '>f8'), ('NGALS', '>i8'), ('N18', '>i8'), \
                   ('N19', '>i8'), ('N20', '>i8'), ('N21', '>i8'), ('N22', '>i8'), ('matching_ID', '<i8'), \
                   ('MEM_MATCH_ID', '<i8'), ('LAMBDA_CHISQ', '<f8'), ('RA_REDM', '<f8'), \
                   ('DEC_REDM', '<f8'), ('q', '<f8'), ('s', '<f8'), ('cosi', '<f8'), ('centered_flag', '<i8')])

print "Change ID from i4 to i8"
print redM_halos.dtype

(numpy.record, [('GALAXY_ID', '>i4'), ('HALOID', '>i4'), ('DESCID', '>i8'), ('RA_OBS', '>f8'), ('DEC_OBS', '>f8'), ('TRA', '>f8'), ('TDEC', '>f8'), ('Z', '>f8'), ('MVIR', '>f8'), ('VMAX', '>f8'), ('VRMS', '>f8'), ('RVIR', '>f8'), ('RS', '>f8'), ('NP', '>f8'), ('PX', '>f8'), ('PY', '>f8'), ('PZ', '>f8'), ('VX', '>f8'), ('VY', '>f8'), ('VZ', '>f8'), ('JX', '>f8'), ('JY', '>f8'), ('JZ', '>f8'), ('SPIN', '>f8'), ('RS_KLYPIN', '>f8'), ('MVIR_ALL', '>f8'), ('M200B', '>f8'), ('M200C', '>f8'), ('M500C', '>f8'), ('M2500C', '>f8'), ('XOFF', '>f8'), ('VOFF', '>f8'), ('SPIN_BULLOCK', '>f8'), ('B_TO_A', '>f8'), ('C_TO_A', '>f8'), ('PID', '>i8'), ('LUMTOT', '>f8'), ('LUM20', '>f8'), ('LBCG', '>f8'), ('NGALS', '>i8'), ('N18', '>i8'), ('N19', '>i8'), ('N20', '>i8'), ('N21', '>i8'), ('N22', '>i8'), ('matching_ID', '<i8'), ('MEM_MATCH_ID', '<i8'), ('LAMBDA_CHISQ', '<f8'), ('RA_REDM', '<f8'), ('DEC_REDM', '<f8'), ('q', '<f8'), ('s', '<f8'), ('cosi', '<f8'), ('centered_flag', '<i8')])
Change ID from i4 to

## Import the BCG catalog with correct ID. 

In [13]:
BCG_filename = datadir + 'v1.6/BCG_v1.6_info.data' 
BCG_file = open(BCG_filename, "rb" )
BCG_data = pickle.load(BCG_file)
BCG_data = BCG_data.view(np.recarray)
print BCG_data.dtype

(numpy.record, [('ID', '<i8'), ('HALOID', '<i8'), ('RA', '<f4'), ('TRA', '<f4'), ('DEC', '<f4'), ('TDEC', '<f4'), ('Z', '<f4')])
[  16116524   16117071   16116541 ... 1008003775 1005616374 1005625859]


## Match BCG_v1.6_info with fullmatchingparams catalog using (RA, DEC, Z)

#### Create function for proximity matching and testing. 

In [24]:
# Finding mismatch between RA, DEC, Z in halorun versus BCG_galaxy
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
from astropy.coordinates import SkyCoord

#Using find_nearest iterate through the array and find index of matched ones, also index of unmatched. 
#Go through the richness-mass template finder. 
BCG_coord = SkyCoord(BCG_data.TRA*u.deg, BCG_data.TDEC*u.deg,  frame='icrs')
halos_coord = SkyCoord(redM_halos.TRA*u.deg, redM_halos.TDEC*u.deg, frame='icrs')
print "Completed SkyCoord"

#Finding nearest neighbor to the catalog.
def find_nearest(ind):
    dist_sep = BCG_coord.separation(halos_coord[ind])
    idx = (np.abs(dist_sep)).argmin()
    return idx, dist_sep[idx].value

#Testing match of specific halo using index testn
testarr = np.arange(0,1000,200)
for i, testn in enumerate(testarr):
    ind, dist = find_nearest(testn)
    print "BCG index: ", testn
    print "Halo matched index:", ind
    print "Total angular separation: ", dist
    print "RA difference: ", redM_halos.TRA[testn] -  BCG_data.TRA[ind]
    print "DEC difference: ", redM_halos.TDEC[testn] - BCG_data.TDEC[ind]
    #print hrun_data.Z[testn] - redM_gal_data.Z[ind] #Some error in redshift
    print "Redshift difference: {:2g} \n".format(BCG_data.Z[ind] - redM_halos.Z[testn]) #Small error in redshift

Completed SkyCoord
BCG index:  0
Halo matched index: 5480575
Total angular separation:  3.41509462487e-06
RA difference:  0.0
DEC difference:  0.0
Redshift difference:  0 

BCG index:  200
Halo matched index: 8388744
Total angular separation:  3.41509462487e-06
RA difference:  0.0
DEC difference:  0.0
Redshift difference:  0 

BCG index:  400
Halo matched index: 6711432
Total angular separation:  5.12264205099e-06
RA difference:  0.0
DEC difference:  0.0
Redshift difference:  0 

BCG index:  600
Halo matched index: 7438583
Total angular separation:  0.0
RA difference:  0.0
DEC difference:  0.0
Redshift difference:  0 

BCG index:  800
Halo matched index: 2071278
Total angular separation:  0.0
RA difference:  0.0
DEC difference:  0.0
Redshift difference:  0 



In [29]:
## For each matched halo in fullmatchingparams replace the GALAXY_ID with ID from BCG_data
#Finding BCG's nearest halo neighbor.

BCG_matched_ID = [] 
halo_matched_ID = []

#Find the centered ones
ang_lim = 1e-5
z_lim = 0.001
BCG_coord = SkyCoord(BCG_data.TRA*u.deg, BCG_data.TDEC*u.deg,  frame='icrs')
halos_coord = SkyCoord(redM_halos.TRA*u.deg, redM_halos.TDEC*u.deg, frame='icrs')

num_matched = 0
for i, cur_halo in enumerate(redM_halos):
    ind, ang_sep = find_nearest(i)
    z_sep = np.abs(cur_halo.Z - BCG_data.Z[ind])

    if (ang_sep <= ang_lim) & (z_sep <= z_lim): 
        cur_halo.GALAXY_ID = BCG_data[ind].ID
        cur_halo.HALOID = BCG_data[ind].HALOID
        num_matched += 1
        
    if i % 1000 == 0: 
        print "Processing redM_halo number ", i
        print "Number of centered are", num_matched
print "Total number of matched redM_halos are", num_matched
print "Out of total number of redM_halos of ", len(halos_coord)


Processing redM_halo number  0
Number of centered are 1
Processing redM_halo number  20
Number of centered are 21
Processing redM_halo number  40
Number of centered are 41
Processing redM_halo number  60
Number of centered are 61
Processing redM_halo number  80
Number of centered are 81
Processing redM_halo number  100
Number of centered are 101
Processing redM_halo number  120
Number of centered are 121
Processing redM_halo number  140
Number of centered are 141
Processing redM_halo number  160
Number of centered are 161
Processing redM_halo number  180
Number of centered are 181
Processing redM_halo number  200
Number of centered are 201
Processing redM_halo number  220
Number of centered are 221
Processing redM_halo number  240
Number of centered are 241
Processing redM_halo number  260
Number of centered are 261
Processing redM_halo number  280
Number of centered are 281
Processing redM_halo number  300
Number of centered are 301
Processing redM_halo number  320
Number of centered 

## Save new redM_halos onto a .data pickle file. 

In [30]:
print redM_halos.GALAXY_ID

[ 90002853535 376001981071 381001274476 ...  47001382759 369001366706
   6003053820]


In [31]:
filehandler = open("redM_halos_fullmatchingparams_matchingv1.3_Buzzardv1.6.data","wb")
pickle.dump(redM_halos, filehandler)
filehandler.close()
print "Done"

Done
